In [1]:
!pip install -qU transformers torch accelerate bitsandbytes
!pip install -q flask
!pip install -q pyngrok

[]

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load Phi-3-mini Instruct model
model_name = "microsoft/phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

In [5]:


# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.7,
    "do_sample": True,
}

# --- Flask App Configuration ---
app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    try:
        data = request.get_json()
        prompt = data.get('prompt')

        if not prompt:
            return jsonify({"error": "Prompt is required"}), 400

        # Construct the chat message format required by Phi-3
        messages = [
            {"role": "user", "content": prompt},
        ]

        # Generate the response
        output = pipe(messages, **generation_args)
        response_text = output[0]['generated_text']

        return jsonify({"response": response_text})

    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

# --- ngrok Tunneling ---
# Sign up for a free ngrok account at https://dashboard.ngrok.com/signup
# and get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "Auth_token" # Replace with your token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok when the app is run
public_url = ngrok.connect(5000) # Port 5000 is the default for Flask
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Start the Flask app in a separate thread
threading.Thread(target=lambda: app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)).start()

Device set to use cuda:0


 * ngrok tunnel "NgrokTunnel: "https://nonappealable-leola-inclinatorily.ngrok-free.dev" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
